Será criado um script para passar os dados de um banco de dados sqlite para um delta table, utilizando spark

* Para iniciar o pyspark no console* :  pyspark --packages io.delta:delta-core_2.12:2.1.0 --driver-class-path ~/sqlite-jdbc-3.34.0.jar
OU
Posso fazer como no script e colocar diretamente no config da SparkSession, assim posso chamar o pyspark ou jupyter notebook sem parâmetros.
* Documentação com exemplo de utilização do jdbc drive para sqlite:*https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/from_to_dbms.html*

In [1]:
 # import libraries
from delta.tables import *
from pyspark.sql import SparkSession
# from pyspark import SparkConf
# from pyspark.sql import *
import pyspark
from delta import *

In [2]:


# Adicionado um outro caminho para o warehouse
builder = SparkSession.builder.appName("Sqlite para Spark") \
.config("spark.sql.warehouse.dir", '/home/fovr/PROJETOS/olist-ml-models/src') \
.config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \
.config("spark.jars", "/home/fovr/PROJETOS/olist-ml-models/jdbc/sqlite-jdbc-3.34.0.jar")  \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.enableHiveSupport() 

spark = configure_spark_with_delta_pip(builder).getOrCreate()



:: loading settings :: url = jar:file:/opt/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fovr/.ivy2/cache
The jars for the packages stored in: /home/fovr/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-640b21d8-4a21-4968-bec3-4fc863882500;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 167ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |

23/04/08 17:07:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
%%markdown
# Configurar e ler tabela sqlite
Será passado para um um dataframe cada tabela do banco de dados.

# Configurar e ler tabela sqlite
Será passado para um um dataframe cada tabela do banco de dados.


In [3]:
url = 'jdbc:sqlite:/home/fovr/PROJETOS/olist-ml-models/db/olist.db'
driver = 'org.sqlite.JDBC'

In [4]:
# Tabela: avaliacao_pedido
df_avaliacao = spark.read.format('jdbc') \
        .options(driver=driver, dbtable='avaliacao_pedido', url=url)\
        .load()

In [5]:
df_avaliacao.show(1)

+--------------------+--------------------+------+--------------------+----------------------+-----------+----------+
|         idAvaliacao|            idPedido|vlNota|descTituloComentario|descMensagemComentario|dtAvaliacao|dtResposta|
+--------------------+--------------------+------+--------------------+----------------------+-----------+----------+
|7bc2406110b926393...|73fc7af87114b3971...|   4.0|                null|                  null| 2018-01-18|2018-01-18|
+--------------------+--------------------+------+--------------------+----------------------+-----------+----------+
only showing top 1 row



In [6]:
# Tabela: cliente
df_cliente = spark.read.format('jdbc') \
        .options(driver=driver, dbtable='cliente', url=url)\
        .load()

In [7]:
df_cliente.show(1)

+--------------------+--------------------+------+----------+------+
|           idCliente|      idClienteUnico|codCep|descCidade|descUF|
+--------------------+--------------------+------+----------+------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...| 14409|    franca|    SP|
+--------------------+--------------------+------+----------+------+
only showing top 1 row



In [8]:
# Tabela: geolocalizacao
df_geolocalizacao = spark.read.format('jdbc') \
        .options(driver=driver, dbtable='geolocalizacao', url=url)\
        .load()

In [9]:
df_geolocalizacao.show(1)

+------+----------+-----------+----------+------+
|codCep|nrLatitude|nrLongitude|descCidade|descUF|
+------+----------+-----------+----------+------+
|  1037|      null|       null| sao paulo|    SP|
+------+----------+-----------+----------+------+
only showing top 1 row



In [10]:
# Tabela: item_pedido
df_item_pedido = spark.read.format('jdbc') \
        .options(driver=driver, dbtable='item_pedido', url=url)\
        .load()

In [11]:
df_item_pedido.show(1)

+--------------------+------------+--------------------+--------------------+-------------+-------+-------+
|            idPedido|idPedidoItem|           idProduto|          idVendedor|dtLimiteEnvio|vlPreco|vlFrete|
+--------------------+------------+--------------------+--------------------+-------------+-------+-------+
|00010242fe8c5a6d1...|           1|4244733e06e7ecb49...|48436dade18ac8b2b...|   2017-09-19|   58.9|  13.29|
+--------------------+------------+--------------------+--------------------+-------------+-------+-------+
only showing top 1 row



In [12]:
# Tabela: pagamento_pedido
df_pagamento_pedido = spark.read.format('jdbc') \
        .options(driver=driver, dbtable='pagamento_pedido', url=url)\
        .load()

In [13]:
df_pagamento_pedido.show(1)

+--------------------+-----------------+-----------------+----------+-----------+
|            idPedido|idPagamentoPedido|descTipoPagamento|nrParcelas|vlPagamento|
+--------------------+-----------------+-----------------+----------+-----------+
|b81ef226f3fe1789b...|                1|      credit_card|         8|      99.33|
+--------------------+-----------------+-----------------+----------+-----------+
only showing top 1 row



In [14]:
# Tabela: pedido
df_pedido = spark.read.format('jdbc') \
        .options(driver=driver, dbtable='pedido', url=url)\
        .load()

In [15]:
df_pedido.show(1)

+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|            idPedido|           idCliente|descSituacao|  dtPedido|dtAprovado|   dtEnvio|dtEntregue|dtEstimativaEntrega|
+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|2017-10-02|2017-10-02|2017-10-04|2017-10-10|         2017-10-18|
+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
only showing top 1 row



In [16]:
# Tabela: produto
df_produto = spark.read.format('jdbc') \
        .options(driver=driver, dbtable='produto', url=url)\
        .load()

In [17]:
df_produto.show(1)

+--------------------+-------------+-------------+------------------+-------+------------+---------------+----------+-----------+
|           idProduto|descCategoria|nrTamanhoNome|nrTamanhoDescricao|nrFotos|vlPesoGramas|vlComprimentoCm|vlAlturaCm|vlLarguraCm|
+--------------------+-------------+-------------+------------------+-------+------------+---------------+----------+-----------+
|1e9e8ef04dbcff454...|   perfumaria|         40.0|             287.0|    1.0|       225.0|           16.0|      10.0|       14.0|
+--------------------+-------------+-------------+------------------+-------+------------+---------------+----------+-----------+
only showing top 1 row



In [18]:
# Tabela: vendedor
df_vendedor = spark.read.format('jdbc') \
        .options(driver=driver, dbtable='vendedor', url=url)\
        .load()

In [19]:
df_vendedor.show(1)

+--------------------+-------+----------+------+
|          idVendedor|descCep|descCidade|descUF|
+--------------------+-------+----------+------+
|3442f8959a84dea7e...|  13023|  campinas|    SP|
+--------------------+-------+----------+------+
only showing top 1 row



In [8]:
%%markdown
# Ingestão - Criando delta table
### Silver

# Ingestão - Criando delta table
### Silver


In [9]:
path = '/home/fovr/PROJETOS/olist-ml-models/delta/silver'

In [23]:
# Ler o dataframe e realizar a ingestão em um delta table


df_avaliacao.write.mode('overwrite').format('delta').partitionBy('dtAvaliacao').save(f'{path}/avaliacao')
df_cliente.write.mode('overwrite').format('delta').partitionBy('descUF').save(f'{path}/cliente')
df_geolocalizacao.write.mode('overwrite').format('delta').partitionBy('descUF').save(f'{path}/geo')
df_item_pedido.write.mode('overwrite').format('delta').save(f'{path}/item_pedido')
df_pagamento_pedido.write.mode('overwrite').format('delta').save(f'{path}/pagamento')
df_pedido.write.mode('overwrite').format('delta').partitionBy('dtPedido').save(f'{path}/pedido')
df_produto.write.mode('overwrite').format('delta').partitionBy('descCategoria').save(f'{path}/produto')
df_vendedor.write.mode('overwrite').format('delta').partitionBy('descUF').save(f'{path}/vendedor')



In [10]:
spark.read.parquet(f'{path}/avaliacao').show(5)

+--------------------+--------------------+------+--------------------+----------------------+----------+-----------+
|         idAvaliacao|            idPedido|vlNota|descTituloComentario|descMensagemComentario|dtResposta|dtAvaliacao|
+--------------------+--------------------+------+--------------------+----------------------+----------+-----------+
|5bf1a70143253c588...|ab8c4bf0204081a45...|   5.0|                null|                  null|2017-12-19| 2017-12-19|
|f5eb912253261b8bb...|dccf99a9f12d7217c...|   4.0|                null|                  null|2017-12-20| 2017-12-19|
|608e890852fc78c38...|2d1f07a3af3df1ddc...|   3.0|                null|  Até agora não rec...|2017-12-21| 2017-12-19|
|875e5a568b6d64e86...|99b1cf2d12de43d08...|   5.0|                null|                  null|2017-12-19| 2017-12-19|
|5fbdc041e09387736...|6c6803a80161b45ad...|   5.0|                null|  Produto original,...|2017-12-20| 2017-12-19|
+--------------------+--------------------+------+------

In [3]:
spark.sql(""" SHOW DATABASES """).show()

23/04/08 17:08:08 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/04/08 17:08:08 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/04/08 17:08:10 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/04/08 17:08:10 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore fovr@127.0.1.1
+---------+
|namespace|
+---------+
|  default|
|   silver|
+---------+



In [12]:
spark.sql(""" USE silver """).show()

23/04/08 16:37:56 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
++
||
++
++



In [25]:
# spark.sql(""" DROP DATABASE silver""").show()
# spark.sql(""" CREATE DATABASE silver""").show()

++
||
++
++



23/04/08 00:02:38 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
23/04/08 00:02:38 WARN ObjectStore: Failed to get database silver, returning NoSuchObjectException


In [13]:
spark.sql(f""" CREATE TABLE IF NOT EXISTS silver.tbAvaliacao USING DELTA LOCATION  "{path}/avaliacao" """).show()
spark.sql(f""" CREATE TABLE IF NOT EXISTS silver.tbCliente USING DELTA LOCATION  "{path}/cliente" """).show()
spark.sql(f""" CREATE TABLE IF NOT EXISTS silver.tbGeolocalizacao USING DELTA LOCATION  "{path}/geo" """).show()
spark.sql(f""" CREATE TABLE IF NOT EXISTS silver.tbItemPedido USING DELTA LOCATION  "{path}/item_pedido" """).show()
spark.sql(f""" CREATE TABLE IF NOT EXISTS silver.tbPagamentoPedido USING DELTA LOCATION  "{path}/pagamento" """).show()
spark.sql(f""" CREATE TABLE IF NOT EXISTS silver.tbPedido USING DELTA LOCATION  "{path}/pedido" """).show()
spark.sql(f""" CREATE TABLE IF NOT EXISTS silver.tbProduto USING DELTA LOCATION  "{path}/produto" """).show()
spark.sql(f""" CREATE TABLE IF NOT EXISTS silver.tbVendedor USING DELTA LOCATION  "{path}/vendedor" """).show()

++
||
++
++

++
||
++
++

++
||
++
++

++
||
++
++

++
||
++
++

++
||
++
++

++
||
++
++

++
||
++
++



In [14]:
spark.sql(""" SHOW TABLES """).show()

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|   silver|      tbavaliacao|      false|
|   silver|        tbcliente|      false|
|   silver| tbgeolocalizacao|      false|
|   silver|     tbitempedido|      false|
|   silver|tbpagamentopedido|      false|
|   silver|         tbpedido|      false|
|   silver|        tbproduto|      false|
|   silver|       tbvendedor|      false|
+---------+-----------------+-----------+



In [15]:
df = spark.sql("SELECT * FROM silver.tbAvaliacao")
df.show(5)

23/04/08 16:38:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+------+--------------------+----------------------+-----------+----------+
|         idAvaliacao|            idPedido|vlNota|descTituloComentario|descMensagemComentario|dtAvaliacao|dtResposta|
+--------------------+--------------------+------+--------------------+----------------------+-----------+----------+
|a36150969fe6bd09e...|ef1b29b591d31d57c...|   1.0|                null|  Demora na entrega...| 2016-11-02|2016-11-03|
|522f795c7ef411740...|25f680bf746bec3df...|   5.0|                null|  Produto conforme ...| 2016-11-02|2016-11-07|
|dd065231d3137dfcb...|48c43415ea1c4af9a...|   3.0|                null|  gostei do produto...| 2016-11-02|2016-11-03|
|58530d17ffb83e571...|f3f12fc90564a9b03...|   3.0|                null|  Entrega muito dem...| 2016-11-02|2016-11-05|
|8b8ec505c06d139a8...|8729a8f7c27d03afe...|   5.0|                null|  Recebi o produto ...| 2016-11-02|2016-11-05|
+--------------------+--------------------+------+------

In [44]:
df = spark.sql("SELECT * FROM silver.tbcliente")
df.show(5)

+--------------------+--------------------+------+--------------------+------+
|           idCliente|      idClienteUnico|codCep|          descCidade|descUF|
+--------------------+--------------------+------+--------------------+------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...| 14409|              franca|    SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|  9790|sao bernardo do c...|    SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|  1151|           sao paulo|    SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|  8775|     mogi das cruzes|    SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...| 13056|            campinas|    SP|
+--------------------+--------------------+------+--------------------+------+
only showing top 5 rows

